# Effective Embedding of Integer Linear Inequalities for Variational Quantum Algorithms

## Notebook for Evaluation

This notebook provides the evaluation code used for quantum algorithms for optimization problems involving linear inequalities. Specifically, we consider the multi-knapsack problem as a test case and provide complete code and dataset, tested for the following quantum algorithms
 - QAOA on QUBO formulations with and without slack-bits
 - Trotterized Adiabatic Evolution (TAE) on QUBO formulations with and without slack-bits
 - variational-TAE, where we optimize the annealing time
 
The code provided was used to generate the results presented in the paper <https://arxiv.org/abs/2403.18395>.

In [1]:
# -*- coding: utf-8 -*-
"""
@author: QUTAC Quantum

SPDX-FileCopyrightText: 2024 QUTAC

SPDX-License-Identifier: Apache-2.0

"""

"""
dimod==0.12.12
ipykernel==6.25.2
matplotlib==3.8.0
pip==23.2.1
PuLP==2.7.0
pyqubo==1.4.0
qiskit==0.44.1
qiskit-aer==0.12.2
"""

'\ndimod==0.12.12\nipykernel==6.25.2\nmatplotlib==3.8.0\npip==23.2.1\nPuLP==2.7.0\npyqubo==1.4.0\nqiskit==0.44.1\nqiskit-aer==0.12.2\n'

# Multi-Knapsack Problem

We first present the mathematical model for the multi-knapsack optimization problem. Given $N$ items and $M$ knapsacks, the aim of the problem is to assign as many items to each knapsack while not exceeding the capacity of any knapsack. Each item can be assigned to maximally one knapsack (or not assigned at all) and contributes its value in this case. The goal is to maximize the total value of all assigned items.

Formally, we can state the problem in the following manner. Let, $ v_{i,j}$ be the value of item $j$, $j \in \{0,1,\dots, N-1\}$ when assigned to knapsack $i$, $w_j$ be the weight of item $j$, and $c_i$ be the capacity of knapsack $i$, $i\in \{0,1,\dots, M-1\}$. We can define a decision variable $x_{i,j}$, such that
$$
\begin{equation*}
    x_{i,j}=
    \begin{cases}
      1, & \text{if item $j$ is assigned to knapsack $i$}\;, \\
      0, & \text{otherwise}\;.
    \end{cases}
  \end{equation*}
$$


The multi-knapsack problem can be formulated as,

$$
\begin{equation*}
\begin{split}
\max &\sum_{i=0}^{M-1} \sum_{j=0}^{N-1} v_{i,j} \cdot x_{i,j} \\
\text{such that } & \sum_{j=0}^{N-1} w_j \cdot x_{i,j} \leq c_i, \hspace{1em} \forall i \in \{0,1,\dots, M-1\}\\
& \sum_{i=0}^{M-1} x_{i,j} \leq 1, \hspace{2em} \forall j \in \{0,1,\dots, N-1\} \\
\end{split}
\end{equation*}
$$

## QUBO Formulation

### 1) Any item $i$ can be assigned to a maximum one knapsack. It is possible that an item is not assigned to any knapsack
$$
H_{\text{single}}= \sum_{j=0}^{N-1} \left(\sum_{i=0}^{M-1} x_{i,j}\right)\cdot \left(\sum_{i=0}^{M-1} x_{i,j}-1 \right) \;.
$$

### 2) Ensure that the capacity of any knapsack is not exceeded. This is achieved by introducing slack bits
$$
\text{H}_{\text{capacity}}=\sum_{i=0}^{M-1} \left[\left(\sum_{j=0}^{N-1} w_j\cdot x_{i,j}\right) + \left(\sum_{b=0}^{\lfloor\log{c_i}\rfloor} 2^{b}\cdot y_{i,b}\right) - c_i \right]^2 \;.
$$

### 3) Maximize the total value of assigned items in knapsacks

$$
H_{\text{obj}}= - \sum_{i=0}^{M-1} \sum_{j=0}^{N-1} v_{i,j} \cdot x_{i,j} \;.
$$

Finally, the complete QUBO is written as,

$$
H = A\cdot H_{\text{single}} + B\cdot \text{H}_{\text{capacity}} + C \cdot H_{\text{obj}} \;, \\
$$

where $A,B$ are the penalty coefficients, and $C$ is the objective weight. Note, $A>0, B>0, C >0$.


In [2]:
%load_ext autoreload
%autoreload 2

# Load Problem Data

We first load the data for problem instances saved in a json file `data/final_knapsack_data.json`. We benchmark the studied algorithms on $20$ different problem instances with increasing problem sizes. 

In [3]:
import json
import numpy as np

file         = 'data/final_knapsack_data.json'
with open(file, 'r') as f:
    dataset = json.load(f)

# QAOA

The code cell below provides the solver for the Quantum Approximate Optimization Algorithm on different QUBO formulations explained in <arXiv.link>. For the classical optimization, we utilize `adam` optimizer. More details of the implementation can be found in the `QAOASolver` class provided in the source code.

In [4]:
import importlib
import sources.QAOAOptimizer
importlib.reload(sources.QAOAOptimizer)
from sources.QAOAOptimizer import QAOASolver


scenario_num     = 2
num_layers       = 3
shots_scale      = 'linear' # 'linear', 'quadratic', 'constant'
qubo_formulation = 'no_slack_qubo' # ['standard_qubo', "no_slack_qubo"]
opt_method       = 'expval' # 'expval', 'cVaR', 'min'
schedule         = "sinusoidal_0.75" # 'sinusoidal_0.75' "sinusoidal_1" "random"
adam_optimizer   = "clip" # "clip" "full"
eps              = 0.1
learning_rate    = 0.01
total_iterations = 500
convl            = 10


scenario = f'scenario_{scenario_num}'
article_value = np.array(dataset[scenario]['article_reward'])
article_weight = np.array(dataset[scenario]['article_weight'])
knapsack_capacity = np.array(dataset[scenario]['knapsack_capacity'])
optimal = np.array(dataset[scenario]['optimal_solution'])
(number_of_knapsacks, number_of_articles) = article_value.shape
num_qubits = number_of_knapsacks * number_of_articles


qaoa = QAOASolver(article_weight, knapsack_capacity, article_value, 
                    optimal, num_layers, opt_method, 
                    qubo_formulation, convl, shots_scale, eps, adam_optimizer)
params=[]
gamma0=[]
beta0=[]

"""Initialization as per trotter results with T = dt * p (num_layers)"""
for layer in range(1, num_layers+1):
    if schedule == "sinusoidal_1":
        s_t = np.sin((np.pi/2) * (np.sin((np.pi*layer)/(2*num_layers))**2))**2
        dt=1.0
        gamma0.append(s_t*dt)
        beta0.append((1-s_t)*dt)
    elif schedule == "sinusoidal_0.75":
        s_t = np.sin((np.pi/2) * (np.sin((np.pi*layer)/(2*num_layers))**2))**2
        dt=0.75
        gamma0.append(s_t*dt)
        beta0.append((1-s_t)*dt)
    elif schedule=="random":
        gamma0.append(np.pi*np.random.rand())
        beta0.append(np.pi*np.random.rand())

params.extend(gamma0)
params.extend(beta0)
params = np.asarray(params)

result = qaoa.optimize_qaoa_knapsack(params, learning_rate, total_iterations)
sampled_states = qaoa.final_samples

if qubo_formulation == "no_slack_qubo":
    qubo = "no_slack_qubo"
    prob_optimal, num_optimal, prob_optimal_90, num_optimal_90, num_valid=qaoa.compute_final_metrics(sampled_states, qubo)

elif qubo_formulation == "standard_qubo":
    qubo = "standard_qubo"
    prob_optimal, num_optimal, prob_optimal_90, num_optimal_90, num_valid=qaoa.compute_final_metrics(sampled_states, qubo)

    qubo = "no_slack_qubo"
    prob_optimal_eval_x, num_optimal_eval_x, prob_optimal_90_eval_x, num_optimal_90_eval_x, num_valid_eval_x=qaoa.compute_final_metrics(sampled_states, qubo)


gammas = result.x[:num_layers]
betas  = result.x[num_layers:2*num_layers]

if qubo_formulation == "no_slack_qubo":
    print(f"Scenario:\t{scenario_num}")
    print(f"QUBO Formulation:\t{qubo_formulation}")
    print(f'QAOA layers:\t{num_layers}')
    print(f'Initial Gamma angles:\t{gamma0}')
    print(f'Initial Beta angles:\t{beta0}')
    print(f'Optimized Gamma angles:\t{gammas}')
    print(f'Optimized Beta angles:\t{betas}')
    print(f'Probability of optimality:\t{prob_optimal}')
    print(f'Probability of 90%-optimality:\t{prob_optimal_90}')
    print(f'No. of classical iterations:\t{result.nit}')
    print(f'Adam optimizer converged:\t{result.success}')
else:
    print(f"Scenario:\t{scenario_num}")
    print(f"QUBO Formulation:\t{qubo_formulation}")
    print(f'QAOA layers:\t{num_layers}')
    print(f'Initial Gamma angles:\t{gamma0}')
    print(f'Initial Beta angles:\t{beta0}')
    print(f'Optimized Gamma angles:\t{gammas}')
    print(f'Optimized Beta angles:\t{betas}')
    print(f'Probability of optimality:\t{prob_optimal}')
    print(f'Probability of 90%-optimality:\t{prob_optimal_90}')
    print(f'Probability of optimality (evaluate x-bits only):\t{prob_optimal_eval_x}')
    print(f'Probability of 90%-optimality (evaluate x-bits only):\t{prob_optimal_90_eval_x}')
    print(f'No. of classical iterations:\t{result.nit}')
    print(f'Adam optimizer converged:\t{result.success}')


Scenario:	2
QUBO Formulation:	no_slack_qubo
QAOA layers:	3
Initial Gamma angles:	[0.10983495705504462, 0.6401650429449551, 0.75]
Initial Beta angles:	[0.6401650429449554, 0.10983495705504487, 0.0]
Optimized Gamma angles:	[ 0.2998  0.5906  0.6826]
Optimized Beta angles:	[ 0.5078  0.2801  0.1760]
Probability of optimality:	0.11166666666666666
Probability of 90%-optimality:	0.11166666666666666
No. of classical iterations:	30
Adam optimizer converged:	True


# Trotterized Adiabatic Evolution (TAE)

The code cell below provides the solver for Trotterized Adiabatic Evolution, on different QUBO formulations explained in <arXiv.link>. More details of the implementation can be found in the `TrotterizedADB` class provided in the source code.

In [5]:
import importlib
import sources.AdiabaticTrotterization
importlib.reload(sources.AdiabaticTrotterization)
from sources.AdiabaticTrotterization import TrotterizedADB
import warnings
warnings.filterwarnings('ignore')

scenario_num     = 2
num_layers       = 3
shots_scale      = 'linear' # 'linear', 'quadratic', 'constant'
qubo_formulation = 'standard_qubo' # ['standard_qubo', "no_slack_qubo"]
opt_method       = 'expval' # 'expval', 'cVaR', 'min'
schedule         = "sinusoidal_0.75" # 'sinusoidal_0.75' "sinusoidal_1" "random"
convl            = 10


scenario = f'scenario_{scenario_num}'
article_value = np.array(dataset[scenario]['article_reward'])
article_weight = np.array(dataset[scenario]['article_weight'])
knapsack_capacity = np.array(dataset[scenario]['knapsack_capacity'])
optimal = np.array(dataset[scenario]['optimal_solution'])
(number_of_knapsacks, number_of_articles) = article_value.shape
num_qubits = number_of_knapsacks * number_of_articles

qaoa = TrotterizedADB(article_weight, knapsack_capacity, article_value, 
                    optimal, num_layers, opt_method, 
                    qubo_formulation, convl, shots_scale)
params = []
gamma0 = []
beta0  = []

"""Initialization as per trotter results with T = dt * p (num_layers)"""
for layer in range(1, num_layers+1):
    if schedule == "sinusoidal_1": 
        s_t = np.sin((np.pi/2) * (np.sin((np.pi*layer)/(2*num_layers))**2))**2
        dt=1.0
        gamma0.append(s_t*dt)
        beta0.append((1-s_t)*dt)
    elif schedule == "sinusoidal_0.75":
        s_t = np.sin((np.pi/2) * (np.sin((np.pi*layer)/(2*num_layers))**2))**2
        dt=0.75
        gamma0.append(s_t*dt)
        beta0.append((1-s_t)*dt)
    elif schedule == "random":
        gamma0.append(np.pi*np.random.rand())
        beta0.append(np.pi*np.random.rand())

params.extend(gamma0)
params.extend(beta0)
params = np.asarray(params)

sampled_states=qaoa.sample_circuit(gamma0, beta0)


if qubo_formulation == "no_slack_qubo":
    qubo = "no_slack_qubo"
    objective_value, min_sol, min_val = qaoa.objective_function(sampled_states,qubo)
    prob_optimal, num_optimal, prob_optimal_90, num_optimal_90, num_valid=qaoa.compute_final_metrics(sampled_states, qubo)   
elif qubo_formulation == "standard_qubo":
    qubo = "standard_qubo"
    objective_value, min_sol, min_val = qaoa.objective_function(sampled_states,qubo)
    prob_optimal, num_optimal, prob_optimal_90, num_optimal_90, num_valid=qaoa.compute_final_metrics(sampled_states, qubo)
    qubo = "no_slack_qubo"
    prob_optimal_eval_x, num_optimal_eval_x, prob_optimal_90_eval_x, num_optimal_90_eval_x, num_valid_eval_x=qaoa.compute_final_metrics(sampled_states, qubo)


if qubo_formulation == "no_slack_qubo":
    print(f"Scenario:\t{scenario_num}")
    print(f"QUBO Formulation:\t{qubo_formulation}")
    print(f'Trotter layers:\t{num_layers}')
    print(f'Probability of optimality:\t{prob_optimal}')
    print(f'Probability of 90%-optimality:\t{prob_optimal_90}')
else:
    print(f"Scenario:\t{scenario_num}")
    print(f"Algorithm:\t{qubo_formulation}")
    print(f'Trotter layers:\t{num_layers}')
    print(f'Probability of optimality:\t{prob_optimal}')
    print(f'Probability of 90%-optimality:\t{prob_optimal_90}')
    print(f'Probability of optimality (evaluate x-bits only):\t{prob_optimal_eval_x}')
    print(f'Probability of 90%-optimality (evaluate x-bits only):\t{prob_optimal_90_eval_x}')


Scenario:	2
Algorithm:	standard_qubo
Trotter layers:	3
Probability of optimality:	0.01125
Probability of 90%-optimality:	0.01125
Probability of optimality (evaluate x-bits only):	0.03925
Probability of 90%-optimality (evaluate x-bits only):	0.03925


# Variational-TAE
The code cell below provides the solver for variational TAE on different QUBO formulations explained in <arXiv.link>, where we optimize the annealing time. For the classical optimization, we again utilize `adam` optimizer. More details of the implementation can be found in the `VarTrotterizedADB` class provided in the source code.

In [6]:
import importlib
import sources.TrotterAnnealingOpt
importlib.reload(sources.TrotterAnnealingOpt)
from sources.TrotterAnnealingOpt import VarTrotterizedADB
import warnings
warnings.filterwarnings('ignore')


scenario_num     = 2
num_layers       = 3
shots_scale      = 'linear' # 'linear', 'quadratic', 'constant'
qubo_formulation = 'no_slack_qubo' # ['standard_qubo', no_slack_qubo ]
opt_method       = 'expval' # 'expval', 'cVaR', 'min'
adam_optimizer   = "clip" # "clip" "full"
eps              = 0.1
learning_rate    = 0.01
total_iterations = 500
convl            = 10


scenario = f'scenario_{scenario_num}'
article_value = np.array(dataset[scenario]['article_reward'])
article_weight = np.array(dataset[scenario]['article_weight'])
knapsack_capacity = np.array(dataset[scenario]['knapsack_capacity'])
optimal = np.array(dataset[scenario]['optimal_solution'])
(number_of_knapsacks, number_of_articles) = article_value.shape
num_qubits = number_of_knapsacks * number_of_articles



annealing_time = [0.2]

qaoa = VarTrotterizedADB(article_weight, knapsack_capacity, article_value, 
                    optimal, num_layers, opt_method, 
                    qubo_formulation, convl, shots_scale, eps, adam_optimizer)

result = qaoa.optimize_annealing_time(annealing_time, learning_rate, total_iterations)

sampled_states=qaoa.final_samples
if qubo_formulation == "no_slack_qubo":
    qubo = "no_slack_qubo"
    prob_optimal, num_optimal, prob_optimal_90, num_optimal_90, num_valid=qaoa.compute_final_metrics(sampled_states, qubo)

elif qubo_formulation == "standard_qubo":
    qubo = "standard_qubo"
    prob_optimal, num_optimal, prob_optimal_90, num_optimal_90, num_valid=qaoa.compute_final_metrics(sampled_states, qubo)

    qubo = "no_slack_qubo"
    prob_optimal_eval_x, num_optimal_eval_x, prob_optimal_90_eval_x, num_optimal_90_eval_x, num_valid_eval_x=qaoa.compute_final_metrics(sampled_states, qubo)


if qubo_formulation == "no_slack_qubo":
    print(f"Scenario:\t{scenario_num}")
    print(f"QUBO Formulation:\t{qubo_formulation}")
    print(f'Trotter layers:\t{num_layers}')
    print(f'Initial annealing time:\t{annealing_time}')
    print(f'Optimized annealing time:\t{result.x[0]**2}')
    print(f'Probability of optimality:\t{prob_optimal}')
    print(f'Probability of 90%-optimality:\t{prob_optimal_90}')
    print(f'No. of classical iterations:\t{result.nit}')
    print(f'Adam optimizer converged:\t{result.success}')
else:
    print(f"Scenario:\t{scenario_num}")
    print(f"QUBO Formulation:\t{qubo_formulation}")
    print(f'QAOA layers:\t{num_layers}')
    print(f'Initial annealing time:\t{annealing_time}')
    print(f'Optimized annealing time:\t{result.x[0]**2}')
    print(f'Probability of optimality:\t{prob_optimal}')
    print(f'Probability of 90%-optimality:\t{prob_optimal_90}')
    print(f'Probability of optimality (evaluate x-bits only):\t{prob_optimal_eval_x}')
    print(f'Probability of 90%-optimality (evaluate x-bits only):\t{prob_optimal_90_eval_x}')
    print(f'No. of classical iterations:\t{result.nit}')
    print(f'Adam optimizer converged:\t{result.success}')


Scenario:	2
QUBO Formulation:	no_slack_qubo
Trotter layers:	3
Initial annealing time:	[0.2]
Optimized annealing time:	0.09619763477912821
Probability of optimality:	0.014666666666666666
Probability of 90%-optimality:	0.014666666666666666
No. of classical iterations:	20
Adam optimizer converged:	True
